In [1]:
modelname = 'resnet18_erl'

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(f'runs/{modelname}')

In [3]:
from tqdm import tqdm_notebook
import numpy as np
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary as summary_
import torchvision
import torchvision.transforms as transforms

import data_loader
import utils
import calibration
import train as t

In [4]:
trainloader, validloader = data_loader.get_train_valid_loader(data_dir='./data',
                           batch_size=512,
                           augment=True,
                           random_seed=42,
                           valid_size=0.1,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
epochs = 15
today_ymd = utils.today_ymd

In [6]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.fc = nn.Linear(512, len(utils.label_names))
model = model.cuda()
model = model.train()
trunk = nn.DataParallel(model)

Using cache found in /home/hyunsikkim/.cache/torch/hub/pytorch_vision_v0.10.0


In [7]:
calib = calibration.CalibrationLoss()
# criterion = nn.CrossEntropyLoss()
criterion = t.ERL(beta=0.1)
# criterion = t.FocalLoss(gamma=3)
# criterion = t.LabelSmoothingLoss(classes=len(utils.label_names), smoothing=0.2)
optimizer = optim.SGD(trunk.parameters(), lr=.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [8]:
ex = t.Experiments(writer, modelname, today_ymd)

In [9]:
ex.load_data(trainloader, validloader)
ex.load_model(trunk)
ex.load_prerequisite(criterion, optimizer, scheduler)
# ex.load_prerequisite(criterion, optimizer, scheduler, mixup_alpha=0.4)

In [ ]:
ex.run()

___